<a href="https://colab.research.google.com/github/rachit-shah/ADBI-projects/blob/master/Speaker_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [36]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5889 sha256=2bdfbaa22aae8825076d5fce4a52321361f27be5b52783b7d81a5f2a452dc659
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
import os
import glob
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import keras.backend as K
import keras
from python_speech_features import fbank, delta
import operator
from collections import OrderedDict

In [0]:
!cp drive/My\ Drive/voxceleb1/vox1_dev_wav_partaa .
!cp drive/My\ Drive/voxceleb1/vox1_dev_wav_partab .
!cp drive/My\ Drive/voxceleb1/vox1_dev_wav_partac .
!cp drive/My\ Drive/voxceleb1/vox1_dev_wav_partad .

In [0]:
!cat vox1_dev* > vox1_dev_wav.zip

In [0]:
!cp vox1_dev_wav.zip drive/My\ Drive

In [0]:
!rm vox1_dev_wav_partaa
!rm vox1_dev_wav_partab
!rm vox1_dev_wav_partac
!rm vox1_dev_wav_partad


In [0]:
!unzip vox1_dev_wav.zip

In [22]:
!ls wav/id10001/eWIX7sfn-M0

00001.wav


In [23]:
import IPython.display as ipd
ipd.Audio('wav/id10001/eWIX7sfn-M0/00001.wav') # load a local WAV file

In [0]:
sample_rate = 12000

In [0]:
def create_files_dictionary():
  train_path = "wav"
  pattern = "**/*.wav"
  files = glob.glob(os.path.join(train_path, pattern), recursive=True)

  classes = []
  for file in files:
    filename = file.split("/")[-1]
    classes.append(file.split("/")[1])
    
  spk_uniq = np.unique(classes)

  train_dict = defaultdict(list)
  print("Number of Classes" ,len(spk_uniq))

  for i in range(len(classes)):
    train_dict[classes[i]].append(files[i])

  return train_dict,spk_uniq

In [39]:
train_dict, spk_uniq = create_files_dictionary()

Number of Classes 1211


In [0]:
subset_train_dict = OrderedDict(sorted(train_dict.items(), key=lambda x: len(x[1]),reverse=True)[:300])

In [64]:
[len(subset_train_dict[i]) for i in subset_train_dict]

[1002,
 656,
 589,
 570,
 489,
 476,
 468,
 426,
 409,
 401,
 394,
 381,
 377,
 374,
 364,
 363,
 357,
 356,
 347,
 343,
 342,
 339,
 336,
 335,
 332,
 330,
 326,
 317,
 316,
 314,
 311,
 310,
 307,
 307,
 303,
 299,
 297,
 297,
 297,
 295,
 295,
 293,
 291,
 290,
 288,
 287,
 286,
 285,
 283,
 283,
 282,
 280,
 279,
 278,
 276,
 275,
 274,
 272,
 270,
 269,
 268,
 267,
 266,
 265,
 265,
 265,
 263,
 261,
 260,
 259,
 258,
 257,
 254,
 254,
 254,
 253,
 253,
 252,
 250,
 248,
 248,
 247,
 245,
 245,
 245,
 245,
 244,
 243,
 243,
 243,
 239,
 238,
 238,
 236,
 236,
 236,
 236,
 235,
 234,
 234,
 232,
 232,
 232,
 232,
 231,
 231,
 231,
 230,
 229,
 228,
 227,
 226,
 226,
 225,
 224,
 224,
 224,
 224,
 223,
 223,
 223,
 222,
 222,
 222,
 222,
 222,
 222,
 221,
 221,
 221,
 220,
 219,
 218,
 218,
 218,
 217,
 217,
 216,
 216,
 216,
 216,
 215,
 212,
 211,
 211,
 211,
 210,
 210,
 210,
 209,
 209,
 206,
 205,
 205,
 205,
 204,
 204,
 202,
 201,
 201,
 200,
 198,
 198,
 198,
 197,
 197,
 19

In [0]:
import json
with open('subset_files_300.json','w') as f:
  json.dump(subset_train_dict,f,indent=4)

In [0]:
with open('subset_files_300.json','r') as f:
  subset_train_dict = json.load(f)

In [0]:
!mkdir subset

In [77]:
!ls

a.txt  drive  sample_data  subset  subset_files_300.json  vox1_dev_wav.zip  wav


In [0]:
from shutil import copyfile

In [0]:
for classid, files in subset_train_dict.items():
  os.mkdir("subset/"+classid)
  for filepath in files:
    filename = filepath.split("/")[-1]
    copyfile(filepath,"subset/"+classid+"/"+filename)

In [0]:
!zip -r subset.zip subset

  adding: subset/ (stored 0%)
  adding: subset/id10225/ (stored 0%)
  adding: subset/id10225/00010.wav (deflated 18%)
  adding: subset/id10225/00005.wav (deflated 16%)
  adding: subset/id10225/00008.wav (deflated 19%)
  adding: subset/id10225/00013.wav (deflated 19%)
  adding: subset/id10225/00014.wav (deflated 20%)
  adding: subset/id10225/00009.wav (deflated 19%)
  adding: subset/id10225/00017.wav (deflated 17%)
  adding: subset/id10225/00004.wav (deflated 18%)
  adding: subset/id10225/00016.wav (deflated 19%)
  adding: subset/id10225/00002.wav (deflated 19%)
  adding: subset/id10225/00011.wav (deflated 19%)
  adding: subset/id10225/00012.wav (deflated 17%)
  adding: subset/id10225/00015.wav (deflated 16%)
  adding: subset/id10225/00003.wav (deflated 18%)
  adding: subset/id10225/00001.wav (deflated 17%)
  adding: subset/id10225/00006.wav (deflated 21%)
  adding: subset/id10225/00007.wav (deflated 22%)
  adding: subset/id10710/ (stored 0%)
  adding: subset/id10710/00084.wav (deflated